## Preliminary operations

In [6]:
# import main libraries
import json
import requests
import pandas as pd
import numpy as np

In [7]:
# import all datasets
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/incidentes2019.json")
incidents_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones_barrios.json")
distances_stations_ntas_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones.json")
distances_stations_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/barrios.json")
ntas_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/estaciones.json")
stations_db = json.loads(response.text)

In [8]:
# get a dataframe with the distances between stations and neighborhoods (ntas)
df_distance_stations_ntas = pd.read_csv("dataset/distance_stations_neighborhoods.csv", index_col="Unnamed: 0")
# get a dataframe with the distances between stations
df_distance_stations = pd.DataFrame(distances_stations_db)

In [9]:
# define a list with vehicle types
vehicle_types = ["engine", "ladder", "rescue", "squad", "hazardous"]
# define vehicle type quantities
vehicle_quantities = [197, 143, 5, 8, 1]

In [10]:
# create a dictionary to store the neighborhoods serviceability dataframes
dfs_nta_serviceability = {}
# create a dictionary to store the maximum servicability distance for each type of vehicle
serviceability_distances ={}

# iterate all vehicle types
for vehicle_type in vehicle_types:
    # create datframes of the same shape of the nta-station distances 
    dfs_nta_serviceability[vehicle_type] = df_distance_stations_ntas.copy()
    # set all the values in the serviceability dataframe to an initial value equal to True
    dfs_nta_serviceability[vehicle_type].loc[:] = True

    # set the initial serviceability distance to an initial value equal to 0
    serviceability_distances[vehicle_type] = 0

## Function *get_coverage_overlaps*

In [11]:
# define a function to detect the stations with a coverage overlap
def get_coverage_overlaps(df):
    # define an internal function to check if there is a coverage overlap between two stations
    def check_coverage_overlap(coverage_station_1, station_2):
        # get the coverage of the second station
        coverage_station_2 = station_2[station_2].index.tolist()
        # chech if there is a complete overlap between the stations'coverage
        is_coverage_included = all(nta in coverage_station_1 for nta in coverage_station_2)
        
        # return the result
        return is_coverage_included

    # transpose dataframe for checking the stations as rows
    df = df.T

    # create a results list
    results = [False] * len(df)
    # convert the list to a series
    results_series = pd.Series(results, index=df.index)

    # iter each station of the dataframe
    for station_name_1, station_1 in df.iterrows():
        # check if the station wasn't already checked
        if results_series[station_name_1] == False:
            coverage_station_1 = station_1[station_1].index.tolist()
            # iter each station of the dataframe
            for station_name_2, station_2 in df.iterrows():
                # chech if the second station is different from first one, and check if it wasn't already checked
                if (station_name_2 != station_name_1) and (results_series[station_name_2] == False):
                    # check the overlap between the coverage of the two stations
                    is_overlap = check_coverage_overlap(coverage_station_1, station_2)
                    # store the result
                    results_series[station_name_2] = is_overlap 
    
    # return the series with the results
    return results_series

## Function *set_serviceability*

In [12]:
# define a function to set the serviceability of each nta from each station
def set_serviceability(distance_threshold, df_nta_serviceability):
    # interate nta-station distances dataframe
    for (nta, distances) in df_distance_stations_ntas.iterrows():
        # iterate stations' distances from the neighborhood
        for station, distance_st_nta in distances.items():
            # check if the distance between the nta and the station is greater then the threshold
            if distance_st_nta > distance_threshold:
                # set the neighborhood as unservable by that station
                df_nta_serviceability.loc[nta, station] = False
    
    # return the serviceability table ready
    return df_nta_serviceability

## Set the serviceability for each vehicle type

In [13]:
# set some thresholds upper limits (JUST FOR HAVING A FASTER CALCULATION)
threshold_upper_limits = [755, 755, 2661, 2531]
#########################################################################

# iterate all vehicle types and relative quantities
for vehicle_type, vehicle_quantity, serviceability_threshold in zip(vehicle_types[:-1], vehicle_quantities[:-1], threshold_upper_limits):
    # set an initial value for the serviceability areas counter
    serviceability_areas = 0
    # make a copy of the original serviceability dataframe (just for the first while check)
    ntas_serviceability = dfs_nta_serviceability[vehicle_type].copy()

    # iterate until:
    # - a neighborhood cannot be served by any station              OR
    # - a station cannot serve any neighborhood                     OR
    # - the serviceability areas are less then the vehicle quantity
    while ntas_serviceability.any(axis=1).all() and ntas_serviceability.any(axis=0).all() and serviceability_areas <= vehicle_quantity:
        # low the serviceability distance threshold
        serviceability_threshold -= 1

        # get a copy of the original dataframe
        #ntas_serviceability = dfs_nta_serviceability[vehicle_type].copy()
        # set the serviceability based on the threshold
        ntas_serviceability = set_serviceability(serviceability_threshold, ntas_serviceability).copy()

        # get the overlaps series
        overlaps = get_coverage_overlaps(ntas_serviceability)

        # count the non-overlapping areas
        serviceability_areas = (~overlaps).sum()

    # get a new copy of the original dataframe
    ntas_serviceability = dfs_nta_serviceability[vehicle_type].copy()
    # set the serviceability based on the last valid threshold (actual + 1)
    ntas_serviceability = set_serviceability(serviceability_threshold + 1, ntas_serviceability).copy()

    # store the serviceability table
    dfs_nta_serviceability[vehicle_type] = ntas_serviceability
    # store the serviceability distance
    serviceability_distances[vehicle_type] = serviceability_threshold + 1
    # print the serviceability distance
    print(vehicle_type + ": " + str(serviceability_threshold + 1))
    

engine: 754
ladder: 754
rescue: 2660
squad: 2530


In [14]:
# save the serviceability tables
for df in dfs_nta_serviceability:
    dfs_nta_serviceability[df].to_csv("dataset/serviceability/" + df + "_ntas_serviceability.csv")